# Pytorch beginner course: Autograd (Pytorch for gradient computation)

## Summary

- [What is Autograd?](#what-is-autograd)
- [How machines compute derivates](#how-machines-compute-derivates)
- [Gradient calculation](#gradient-calculation)
- [Backward function and the Jacobian matrix](#backward-function-and-the-jacobian-matrix)
- [Gradient history management](#gradient-history-management)
- [Glossary of the used tools](#glossary-of-the-used-tools)
    - [Methods](#methods)
    - [Properties](#properties)
- [References](#references)
- [Author](#author)

## What is autograd?

Machine learning can be considered as an optimization problem, usually a minimization problem, for this reason the **derivates** are the main tools used by the training algorithms, thankfully pytorch offers a specifi module to solve the *derivates* and in particular the *gradients* of the functions, this module is `torch.autograd`.

As we will see, the concept of the derivates is very important to understand the mechanism of a learning algorithms, and with `autograd` we will not waste time to implements manually all functions that calculate the gradients, pytorch is one of the most used deep learning framework also for the `autograd`.

The main pros of the `autograd` are:
1. *Easy to use*
2. *Efficiency*
3. *Flexibility*

## How machines compute derivates

In computer science we have three different ways to calculate derivates:

* Numerical differentiation
* Symbolic differentiation
* Automatic differentiation *(that combine the two previous approaches)*

in this lecture we will not study the theory behind this three approaches, but that's important to know the main pros and cons among them:

| Method | Precision | Velocity | Application |
|:------:|:---------:|:--------:|:-----------:|
| Numerical Differentiation | Approximate | Rapidly | Simple function |
| Symbolic Differentiation | Exactly | Slow | Complex function |
| Automatic Differentiation | Exactly or Approximate | Rapidly | Complex function |

The `autograd` module use the **Automatic differentiation**, which combines the **Numerical** and **Symbolic** approaches

## Gradient calculation

Now we will see in practice how is possible to calculate the gradients with `autograd`.

For the first is necessary to set the attribute `requires_grad=True` to specify that we want track the tensor during the gradients calculation

In [93]:
import torch

x = torch.tensor([2.,3.], requires_grad=True)
y = torch.tensor([4.,2.], requires_grad=True)

print(x)
print(y)

tensor([2., 3.], requires_grad=True)
tensor([4., 2.], requires_grad=True)


Now our tensors are ready to know their gradients during the computation.

For semplicity now we apply a simple operation between this two tensors and we will save the output of this opertation into an another tensor `z`

In [94]:
z = (x**3)+(y**2)

print(z)

tensor([24., 31.], grad_fn=<AddBackward0>)


Now is very importanto to pay attention to the last output, we will see that the our tensor `z` have a particular attribute named `grad_fn` that explain the gradient function that we will use to apply the **backward** method, as we can see the gradient function is named `<AddBackward0>` and this information is very important for us because the name of gradient function explain us that the tensor `z` was born from a sum operation *(the sum between $x^3$ and $y^2$)*.

The name of *gradient function* change if we change the basic operation to obtain the output tensor, as follow we show some of these functions:

| Operation | Gradient Function |
|:---------:|:-----------------:|
| `+` | `<AddBackward>` |
| `-` | `<SubBackward>` |
| `*` | `<MulBackward>` |
| `/` | `<DivBackward>` |
| `mean()` | `<MeanBackward>` |

Essentially, the `grad_fn` contain an object instance pointer of the class `torch.autograd.Function` if the tensor was made by an operation between two tensor, otherwise the `grad_fn` attribute have `None` value.

In [95]:
print("grad_fn of z: ", z.grad_fn)   # z was made by an operation between x and y
print("grad_fn of x: ", x.grad_fn)   # x was made by a user (don't have a gradient function)
print("grad_fn of y: ", y.grad_fn)   # y was made by a user (don't have a gradient function)

grad_fn of z:  <AddBackward0 object at 0x000001E41F8489A0>
grad_fn of x:  None
grad_fn of y:  None


Now we can procede to compute a gradients of `x` and `y`

In [96]:
# backward() method compute the gradients respect to the leaf of the graph (x and y are the leafs in our scenario)
# the backward() method work only on a scalar value, for this reason we must do an operation that compress the output tensor in a scalar value,
# in this case we have used a sum() function, but we could have used any function, for example mean()
z = z.sum()

# Now we can calculate the gradients...
z.backward()

# ...and print the gradients of our input tensor x and y
print("Gradients of x:", x.grad)

print("Gradients of y:", y.grad)

Gradients of x: tensor([12., 27.])
Gradients of y: tensor([8., 4.])


The output obtained is given by the calculation:

$$
\begin{bmatrix}
\frac{\partial z}{\partial x}
\\
\frac{\partial z}{\partial y}
\end{bmatrix}
=
\begin{bmatrix}
3x^2
\\
2y
\end{bmatrix}
=
\begin{bmatrix}
3x_1^2 \quad 3x_2^2
\\
2y_1 \quad 2y_2
\end{bmatrix}
=
\begin{bmatrix}
3 \cdot 2^2 \quad 3 \cdot 3^2
\\
2 \cdot 4 \quad 2 \cdot 2
\end{bmatrix}
=
\begin{bmatrix}
12 \quad 27
\\
8 \quad 4
\end{bmatrix}
$$

Below we can see the representation of our simple computation

![backward concept](images/backward.svg)

## Backward function and the Jacobian matrix

The `backward` using a ***Jacobian matrix*** to compute the gradients, in linear algebra the *Jacobian Matrix* is a particular matrix that contain all partial derivates of a function, in this matrix each column represent tha gradient respect to each variables.

Let’s say we have a vector of functions composed of two functions:

$$
G = \binom{g_1}{g_2} = \binom{y^2+a}{x^2+a}
$$

the Jacobian matrix of our $G$:

$$
J_G = 
\begin{bmatrix}
\frac{\partial g_1}{\partial x_1} \quad \frac{\partial g_1}{\partial y_1}
\\
\frac{\partial g_2}{\partial x_2} \quad \frac{\partial g_2}{\partial y_2}
\end{bmatrix}
=
\begin{bmatrix}
0 \quad 2y
\\
2x \quad 0
\end{bmatrix}
$$

If we specify other tensor *(with same dimension of an input tensors)* in a `backward` function we can compute the Jacobian matrix of our function multiplied by the specified vector:

$$
J \cdot v
$$

In [97]:
# Init two input tensors
x = torch.tensor([2.,3.], requires_grad=True)
y = torch.tensor([3.,4.], requires_grad=True)

# compute an output tensor created by a sum operation
z = (x**2)+(y**3)

# define a tensor 1D with the same dimension of x and y
v = torch.tensor([1.,2.], dtype=torch.float32)

# dz/dx, dz/dy * v
z.backward(v) 

# output of the gradients
print(x.grad)
print(y.grad)

tensor([ 4., 12.])
tensor([27., 96.])


This output can be explained by the below computation:

$$
J_z \cdot v =
\begin{bmatrix}
\frac{\partial z}{\partial x_1} \quad \frac{\partial z}{\partial y_1}
\\
\frac{\partial z}{\partial x_2} \quad \frac{\partial z}{\partial y_2}
\end{bmatrix}
\cdot
\begin{bmatrix}
v_1
\\
v_2
\end{bmatrix}
=
\begin{bmatrix}
4 \quad 27
\\
6 \quad 48
\end{bmatrix}

\begin{bmatrix}
1
\\
2
\end{bmatrix}
=
\begin{bmatrix}
4 \quad 27
\\
12 \quad 96
\end{bmatrix}
$$

## Gradient history management

Keeping track of the computational graph turns out to be very expensive in terms of computational costs, for this reason if we don't need to calculate the gradient the best way is detach the computational graph, as follow we will show you some methods and best practice to manage the gradients history.

In [98]:
x = torch.ones(3, requires_grad=True)
print("x requires_grad: ", x.requires_grad)

# Set requires_grad to False
x.requires_grad_(False)
print("x requires_grad: ",x.requires_grad)    

# copy the x tensor into the y tensor without the tracking of the computational graph
y = x.detach()
print("y requires_grad: ", y.requires_grad)

x requires_grad:  True
x requires_grad:  False
y requires_grad:  False


We can use the `with` python keyword to define a specific block of code that don't track a computational graph

In [99]:
x = torch.ones(3, requires_grad=True)

# y tensor if we track the computational graph
y = x+2
print("y tensor with computational graph: ", y)

# y tensor if we don't track the computational graph
with torch.no_grad():
    y = x+2
    print("y tensor into the 'with' code block: ", y)

y tensor with computational graph:  tensor([3., 3., 3.], grad_fn=<AddBackward0>)
y tensor into the 'with' code block:  tensor([3., 3., 3.])


## Glossary of the used tools

### Methods

- `torch.autograd.backward()`
- `torch.autograd.requires_grad_()`
- `torch.autograd.no_grad()`

### Properties

- `torch.autograd.grad`
- `torch.autograd.requires_grad`
- `torch.autograd.grad_fn`

## References

[Pytorch documentation](https://pytorch.org/docs/stable/index.html)

## Author

Emilio Garzia, 2024

[Github](https://github.com/EmilioGarzia)

[Linkedin](https://www.linkedin.com/in/emilio-garzia-58a934294/)